In [15]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import csv
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

In [16]:
def openFile(file_name):
    with open(file_name, encoding="utf-8") as csvfile: 
        next(csvfile)
        rawArticles = csv.reader(csvfile, delimiter=',') 
        all_reviews = [] 
        all_sentiment = []
        for row in rawArticles:
            all_reviews.append((row[3].lower()).split())
            all_sentiment.append(row[5].lower())
        return all_reviews, all_sentiment

In [17]:
def getStopWordsList(stopwordsfile):
    stopwords=[]
    file_stopwords = open(stopwordsfile,'r')
    row = file_stopwords.readline()
    while row:
        word = row.strip()
        stopwords.append(word)
        row = file_stopwords.readline()
    file_stopwords.close()
    return stopwords

In [18]:
def getEmojiHandling(review):
    emoji = []
    for word in review:
        #Smile -- :), : ), :-), (:, ( :, (-:, :')
        word = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))','POS',word)
        
        #Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
        word = re.sub(r'(:\s?D|:-D|x-?D|X-?D)','POS',word)

        # Sad -- :-(, : (, :(, ):, )-:
        word = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' NEG ', word)

        # Cry -- :,(, :'(, :"(, T_T
        word = re.sub(r'(:,\(|:\'\(|:"\(|T_T)', ' NEG ', word)

        emoji.append(word)
    return emoji

In [19]:
def getPunctHandling(review):
    #menghilangkan tanda baca
    preprocess_review = []
    for word in review:
        word = word.strip('\'"?!,.():;')

        #mengkonversi huruf vocal lebih dari satu dan berurutan
        word_character = re.compile(r"(.)\1+", re.DOTALL)
        word = word_character.sub(r"\1\1", word)

        #menghilangkan tanda - & '
        word = re.sub(r'(-|\')','',word)

        preprocess_review.append(word.lower())
    return preprocess_review

In [20]:
def getStemmingSentence(review):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(review)

In [21]:
def getFeatureVector(review,stop_words_indo,stop_words_eng):
    feature_vector = []
    list_no = ['ga','engga','enggak','gak','nggak','ngga','tdk']
    for word in review:
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", word) #menghilangkan karakter selain huruf didalam kata
        if (word in stop_words_indo or val is None or word in stop_words_eng):
            continue
        else:
            if word in list_no:
                word = 'tidak'
            feature_vector.append(word)
    for_stemming = ' '.join(feature_vector)
    return feature_vector, for_stemming


In [22]:
def getNegativeHandling(review):
    negative_review = []
    for i in range(len(review)):
        word = review[i]
        if review[i-1] != 'tidak':
            negative_review.append(word)
        else:
            word = 'tidak_'+word
            negative_review.append(word)
    return negative_review

In [23]:
def preprocessReview(review):
    return getPunctHandling(getEmojiHandling(review))

In [24]:
def createFreqDict(reviewHandled):
    freqOfWord = {}
    for sentence in reviewHandled:
        for word in sentence:
            if word in freqOfWord:
                freqOfWord[word] += 1
            else:
                freqOfWord[word] = 1
    file_key = open('keys.txt','w')
    for key in freqOfWord.keys():
        file_key.write(str(key))
        file_key.write("\n")
    file_key.close()
    return freqOfWord

In [25]:
def getFeatureExtraction(review):
    words = set(review)
    features = {}
    for word in feature_list.keys():
        features['contains(%s)' % word] = (word in words) 
    return features

In [26]:
all_reviews, all_sentiments = openFile('dataset.csv')
stop_words_indo = getStopWordsList('stopwordsindo.txt')
stop_words_eng = stopwords.words('english')

In [27]:
%%time
preprocess_reviews = []
tokens = []
reviews = []
handled_reviews = []
feature_list = []
for review in all_reviews:
    feature, review_for_stem = getFeatureVector(preprocessReview(review),stop_words_indo,stop_words_eng)
    preprocess_reviews.append(getStemmingSentence(review_for_stem))

Wall time: 20min 2s


In [28]:
%%time
for review in preprocess_reviews:
    tokens.append(nltk.word_tokenize(review))

Wall time: 560 ms


In [29]:
%%time
for i in range(len(tokens)):
    neg_handled_rev = getNegativeHandling(tokens[i])
    handled_reviews.append(neg_handled_rev)
    reviews.append((' '.join(neg_handled_rev),all_sentiments[i]))

Wall time: 8.99 ms


In [38]:
data = []
sentiments = []
for i in range(len(reviews)):
    data.append(reviews[i][0])
    sentiments.append(reviews[i][1])
print(len(data))

500


In [40]:
labels = np.zeros(500)
labels[0:400] = 0
labels[400:500] = 1
kf = StratifiedKFold(n_splits=10)
totalNB = 0
totalMatNB = np.zeros((2,2))
len(labels)

500

In [41]:
for train_idx, test_idx in kf.split(data,sentiments):
    X_train = [data[i] for i in train_idx]
    X_test = [data[i] for i in test_idx]
    y_train,y_test = labels[train_idx], labels[test_idx]
    vectorizer = TfidfVectorizer(min_df=0.0, max_df=1.0, sublinear_tf=True, use_idf=True, stop_words='english')
    X_train_tf_idf = vectorizer.fit_transform(X_train)
    X_test_tf_idf = vectorizer.transform(X_test)

    model = MultinomialNB()
    model.fit(X_train_tf_idf, y_train)
    result = model.predict(X_test_tf_idf)

    totalMatNB = totalMatNB + confusion_matrix(y_test, result)
    totalNB = totalNB + sum(y_test==result)
    print(train_idx,test_idx)
    

print(totalMatNB, totalNB/len(data))

[ 50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175
 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193
 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211
 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229
 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247
 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 294 295 29

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24